<h2>PROGETTO INDIVIDUALE:</h2> 
<h3>APPLICAZIONE IN SPARK CHE, DATO UN INSIEME DI MALATTIE NOTE, ATTRAVERSO TEXT MINING E SENTIMENT ANALYSIS TROVI DA LAVORI SCIENTIFICI ON-LINE DEI GENI ASSOCIATI A TALI MALATTIE</h3>
<i>Salvatore Danilo Palumbo - 0706508</i>

In [ ]:
'''
SETUP ENVIRONMENT FOR GOOGLE COLAB
'''

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-3.0.0-preview2/spark-3.0.0-preview2-bin-hadoop3.2.tgz
!tar -xvf spark-3.0.0-preview2-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install biopython
!rm -rf sample_data/
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-preview2-bin-hadoop3.2"

spark-3.0.0-preview2-bin-hadoop3.2/
spark-3.0.0-preview2-bin-hadoop3.2/data/
spark-3.0.0-preview2-bin-hadoop3.2/data/streaming/
spark-3.0.0-preview2-bin-hadoop3.2/data/streaming/AFINN-111.txt
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/sample_binary_classification_data.txt
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/sample_kmeans_data.txt
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/sample_multiclass_classification_data.txt
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/sample_lda_libsvm_data.txt
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/iris_libsvm.txt
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/pagerank_data.txt
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/sample_linear_regression_data.txt
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/pic_data.txt
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/als/
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/als/test.data
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/als/sample_movielens_rati

In [ ]:
'''
SPARK IMPORTS
'''

import findspark
findspark.init()
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SQLContext  
from pyspark.sql.functions import udf
import pyspark.sql.functions as f
from pyspark.sql.types import StructType, StringType, IntegerType, StructField, ArrayType,DoubleType

In [ ]:
'''
SETUP SPARKCONF
'''

conf = SparkConf().setAppName("Spark")
conf.set("spark.executor.heartbeatInterval","100000")
sc = SparkContext('local[*]', '', conf=conf) # uses 4 cores on your local machine
sql = SQLContext(sc)

In [ ]:
'''
STANDARD IMPORTS
'''

import pandas as pd
import numpy as np
import random
import os
from Bio import Entrez
from Bio import Medline
from Bio import SeqIO
import nltk
from nltk.corpus import stopwords,words,wordnet
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk import ne_chunk
from operator import add


'''
NLTK packages
'''
nltk.download('vader_lexicon')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

La ricerca avviene a partire da 10 differenti malattie, con denominazione in inglese, e per ogni malattia si vanno a ricercare 200 articoli:
* Rosolia
* Ebola
* Tubercolosi
* Morbillo
* Tetano
* Poliomelite
* Parotite / Orecchioni
* Epatite
* Varicella

In [ ]:
MAX_COUNT = 200 ## Numero di articoli per malattia
TERMS = ['Rubella','Ebola','Tuberculosis','Measles','Tetanus','Typhus','Polio','Mumps','Hepatitis','Varicella'] ## Malattie

La funzione <b>search</b> permette di effettuare delle query su <b>PubMed</b> e di restituire i record relativi all'articolo.

Dall'articolo, vengono prelevati il titolo e l'abstract (quando disponibile), poichè nella maggioranza degli articoli non è possibile prelevare direttamente il full text.

In [ ]:
def search(TERM):
    Entrez.email = 'random@example.com'
    handle = Entrez.esearch(db='pubmed', sort='relevance', retmax=MAX_COUNT, retmode='text', term=TERM)
    result = Entrez.read(handle)
    ids = result['IdList']
    handle = Entrez.efetch(db='pubmed', sort='relevance', retmode='text', rettype='medline', id=ids)
    records = Medline.parse(handle)
    return records

Creo una lista contenente le concatenazioni di titolo e abstracts. I tag utilizzati dall'*NCBI* sono riportati [qui](https://www.ncbi.nlm.nih.gov/books/NBK3827/table/pubmedhelp.Tn/).

In [ ]:
titles_abstracts = []

for term in TERMS:
    for record in search(term):
        title = record.get('TI','')
        abstract = record.get('AB','')
        titles_abstracts.append(title+abstract)

A partire dalla lista, creo un DataFrame

In [ ]:
spark_df = sql.createDataFrame(zip(titles_abstracts), schema=['titles_abstracts'])

In [ ]:
spark_df.show()

+--------------------+
|    titles_abstracts|
+--------------------+
|Progress Toward R...|
|Rubella IgM epide...|
|Rubella seroepide...|
|Diagnostic algori...|
|Epidemiology of a...|
|Seroepidemiologic...|
|Rubella seropreva...|
|Towards eliminati...|
|Measles and rubel...|
|Antibodies agains...|
|Rubella in Poland...|
|Seroprevalence of...|
|Immunity to rubel...|
|Rubella virus, To...|
|Seroprevalence of...|
|Progress Toward R...|
|Rubella Virus Inf...|
|WHO international...|
|Biological view o...|
|Molecular aspects...|
+--------------------+
only showing top 20 rows



Come controllo, elimino tutti gli elementi nulli

In [ ]:
spark_df = spark_df.dropna()
spark_df.count()

2000

Utilizzando <b>NLTK</b>, ovvero una libreria di Python per il processamento del linguaggio naturale, vado ad eliminare segni di punteggiatura e stopwords attraverso due funzioni.

In [ ]:
'''
Clean texts
'''

def remove_punctuation(sentence):
    tokenizer = RegexpTokenizer(r'\w+')
    result = tokenizer.tokenize(sentence)
    return result

def remove_stopwords(sentence):
    stop_words = set(stopwords.words('english'))
    text = word_tokenize(sentence)
    new_sentence =[]
    for w in text:
        if w.lower() not in stop_words: new_sentence.append(w)
    return new_sentence

def clean(sentence):
    return ' '.join(remove_stopwords(' '.join(remove_punctuation(sentence))))

Sempre sfruttando **NLTK**, è possibile effettuare sentiment analysis su testo. <br>
Trattandosi di modelli pre-addestrati, i risultati potrebbero variare o non essere pienamente precisi.

<br><br>

Attraverso queste funzioni, è possibile ottenere le varie tipologie di punteggio.

In [ ]:
'''
Get sentiment scores
'''

def get_scores(sentence):
    sid = SentimentIntensityAnalyzer()
    sid.polarity_scores(sentence)
    return {'POSITIVE' : sid.polarity_scores(sentence)['pos'],'NEUTRAL': sid.polarity_scores(sentence)['neu'], 'NEGATIVE': sid.polarity_scores(sentence)['neg'] }


def get_positive_score(sentence):
    sid = SentimentIntensityAnalyzer()
    sid.polarity_scores(sentence)
    return sid.polarity_scores(sentence)['pos']

def get_neutral_score(sentence):
    sid = SentimentIntensityAnalyzer()
    sid.polarity_scores(sentence)
    return sid.polarity_scores(sentence)['neu']


def get_negative_score(sentence):
    sid = SentimentIntensityAnalyzer()
    sid.polarity_scores(sentence)
    return sid.polarity_scores(sentence)['neg']

<h1>TEST SENTIMENT ANALYSIS</h1>
<br>
Come test, vado ad effettuare la pulizia di una frase attraverso le funzioni definite prima e anche della sentiment analysis.

In [ ]:
sentence = 'i hate you, stupid!'

In [ ]:
remove_stopwords(sentence)

['hate', ',', 'stupid', '!']

In [ ]:
remove_punctuation(sentence)

['i', 'hate', 'you', 'stupid']

In [ ]:
clean(sentence)

'hate stupid'

In [ ]:
get_scores(sentence)

{'NEGATIVE': 0.881, 'NEUTRAL': 0.119, 'POSITIVE': 0.0}

Creazione di UDF (User Defined Function), necessarie per poter processare i vari titoli ed abstracts.

In [ ]:
'''
  DEFINING UDFS
'''
clean_udf = udf(clean, StringType())
get_scores_udf = udf(get_scores, StringType())
get_positive_score_udf = udf(get_positive_score,DoubleType())
get_neutral_score_udf = udf(get_neutral_score,DoubleType())
get_negative_score_udf = udf(get_negative_score,DoubleType())

Attraverso le UDF, vado ad aggiungere al DataFrame 4 differenti colonne:
* Colonna contenente titoli ed abstracts puliti
* Punteggi positivi per titoli ed abstracts puliti
* Punteggi neutrali per titoli ed abstracts puliti
* Punteggi negativi per titoli ed abstracts puliti

In [ ]:
spark_df = (
    spark_df
    .withColumn("cleaned_titles_abstracts", clean_udf(spark_df.titles_abstracts))
)

In [ ]:
spark_df = (
    spark_df
    .withColumn("cleaned_titles_abstracts_POSITIVE_score", get_positive_score_udf(spark_df.cleaned_titles_abstracts))
    .withColumn("cleaned_titles_abstracts_NEUTRAL_score", get_neutral_score_udf(spark_df.cleaned_titles_abstracts))
    .withColumn("cleaned_titles_abstracts_NEGATIVE_score", get_negative_score_udf(spark_df.cleaned_titles_abstracts))
)

In [ ]:
spark_df.show(30)

+--------------------+------------------------+---------------------------------------+--------------------------------------+---------------------------------------+
|    titles_abstracts|cleaned_titles_abstracts|cleaned_titles_abstracts_POSITIVE_score|cleaned_titles_abstracts_NEUTRAL_score|cleaned_titles_abstracts_NEGATIVE_score|
+--------------------+------------------------+---------------------------------------+--------------------------------------+---------------------------------------+
|Progress Toward R...|    Progress Toward R...|                                  0.093|                                 0.809|                                  0.098|
|Rubella IgM epide...|    Rubella IgM epide...|                                  0.174|                                  0.78|                                  0.046|
|Rubella seroepide...|    Rubella seroepide...|                                  0.045|                                 0.911|                                  0.044

<h1>TEST SENTIMENT ANALYSIS SU TITLES + ABSTRACTS PULITI</h1>

<br>

Per verificare i punteggi ottenuti, seleziono il massimo per ogni tipo di punteggio applicato su titoli ed abstracts puliti:
questi punteggi variano in base a diversi fattori, come ad esempio la lunghezza del testo, la presenza di determinate keywords "positive", "negative" o "neutrali" ecc.

La maggioranza degli articoli ha come score predominante quello neutrale: trattandosi di modelli pre-addestrati, è sicuramente possibile ottenere migliori risultati andando ad addestrare un classificatore specifico per questo tipo di applicazione.

Lo score ottenuto dipende da numerosi fattori, come la lunghezza del testo, la tipologia delle parole ecc.

In [ ]:
## Max positive score

#spark_df.rdd.max(key=lambda x: x["cleaned_titles_abstracts_POSITIVE_score"])

In [ ]:
## Max neutral score

#spark_df.rdd.max(key=lambda x: x["cleaned_titles_abstracts_NEUTRAL_score"])

In [ ]:
## Max negative score

#spark_df.rdd.max(key=lambda x: x["cleaned_titles_abstracts_NEGATIVE_score"])

<h1>TEXT MINING</h1>

<br>
A questo punto, a partire da titoli ed abstracts puliti, bisogna andare a ricavare le varie malattie. All'interno di ogni articolo, si ha sicuramente un'alta frequenza del nome della malattia in questione:
Pertanto, in una prima fase, si va ad effettuare una <b>word count</b> applicando due ulteriori filtri:
* Rimozione di numeri
* Rimozione di parole con lunghezza minore di 3

In [ ]:
## Frequency Count of distinct words + remove digits and remove words w. len < 3

frequency = spark_df.select("cleaned_titles_abstracts").rdd \
            .flatMap(lambda x: x[0].split(' ')) \
            .map(lambda x: (x, 1)) \
            .filter(lambda x: x[0].isdigit() == False) \
            .filter(lambda x: len(x[0]) > 3) \
            .reduceByKey(lambda x,y: x+y) \
            .sortBy(ascending=False, keyfunc=lambda key_value: key_value[1]) \
            .collect()

In [ ]:
frequency

[('vaccine', 2480),
 ('vaccination', 2201),
 ('measles', 2174),
 ('rubella', 1785),
 ('cases', 1747),
 ('virus', 1590),
 ('mumps', 1412),
 ('study', 1399),
 ('varicella', 1373),
 ('years', 1370),
 ('patients', 1282),
 ('infection', 1262),
 ('children', 1218),
 ('polio', 1130),
 ('among', 1096),
 ('disease', 1087),
 ('tuberculosis', 1078),
 ('dose', 957),
 ('outbreak', 925),
 ('hepatitis', 909),
 ('Ebola', 835),
 ('tetanus', 825),
 ('immunization', 793),
 ('data', 789),
 ('health', 785),
 ('high', 761),
 ('risk', 722),
 ('countries', 697),
 ('incidence', 697),
 ('using', 680),
 ('antibody', 675),
 ('population', 661),
 ('associated', 661),
 ('group', 644),
 ('reported', 642),
 ('immunity', 629),
 ('RESULTS', 602),
 ('outbreaks', 592),
 ('METHODS', 588),
 ('case', 581),
 ('antibodies', 576),
 ('typhus', 574),
 ('used', 557),
 ('clinical', 551),
 ('vaccines', 543),
 ('women', 531),
 ('analysis', 526),
 ('rate', 525),
 ('year', 516),
 ('months', 514),
 ('specific', 512),
 ('transmission', 

Analizzando le frequenze, quello che si può notare è che effettivamente i nomi delle malattie sono abbastanza frequenti; tuttavia sono presenti anche altre parole molto più frequenti che sono legate alla malattia, come ad esempio "vaccination", "infection" ecc.

<br>

Una possibile soluzione è quella di creare un classificatore che permetta di classificare una parola come malattia o meno; in questo caso, vado ad utilizzare un dizionario contenente parole particolarmente ricorrenti:
Nel dettaglio, questo dizionario viene "espanso" attraverso due operazioni, ovvero <b>Stemming</b> e <b>Lemmatization</b>.

<br>

Entrambe le operazioni permettono di normalizzare le parole nella loro forma base, come ad esempio waiting -> wait.

La differenza tra <b>stemming</b> e <b>lemmatization</b> è che lemmatization considera il contesto e converte le parole nella loro base significativa, mentre lo stemming rimuove solo gli ultimi caratteri,
portando anche ad alcuni errori.
    
Ad esempio la lemmatization potrebbe identificare la forma base di "caring" come "care", mentre lo stemming va a rimuovere "-ing", trasformando la parola in "car".

Anche nel caso in cui lo stemming commetta degli errori è irrilevante, in quanto si va solamente a verificare se la parola presente all'interno del dizionario è anche presente all'interno dei vari titoli ed abstract.



In [ ]:
'''
DICTIONARY CREATION
'''

lem = WordNetLemmatizer()
stem = PorterStemmer()

dictionary = ['vaccines','viruses','patients','risks','studies','data','diseases','methods','humans','antibodies','positive','negative','high','low','infections','health','children','vaccination','cases','years','outbreak',
              'doses','China','USA','treatments','analysis','transmissions','population','immunization','results','countries','incidence','anti','groups','outbreaks','toxoid','time','rate','immune','immunity','response',
               'controls', 'backgrounds','levels','women','men'
]

Questa trasformazione applica lemmatization e stemming ad ogni parola del dizionario, selezionando solamente valori distinti: <br>
in particolare, al dizionario iniziale vengono effettuate due *map* che effettuano il lemmatization e lo stemming di ogni parola; dopodichè vengono effettuate due union per concatenare il tutto.

In [ ]:
dictionary = sc.parallelize(dictionary) \
    .map(lambda x: lem.lemmatize(str(x))) \
    .union(sc.parallelize(dictionary).map(lambda x: stem.stem(str(x)))) \
    .union(sc.parallelize(dictionary)) \
    .distinct() \
    .collect()


A questo punto, sempre attraverso NLTK, vado ad effettuare il **POS** tagging, ossia il *Part-Of-Speech* tagging, ovvero una tecnica utilizzata per assegnare parti del discorso a ciascuna parola di un determinato testo (come sostantivi, verbi, pronomi, avverbi, congiunzioni, aggettivi, ecc.) in base alla sua definizione e al suo contesto.
<br>
Per affinare la ricerca, vado a considerare solamente i <b>Nouns</b> (sostantivi). La lista dei tag del Part-Of-Speech è riportata [qui](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html)

Per effettuare il POS tagging, bisogna suddividere il testo in <b>tokens</b>: questa operazione viene effettuata con la prima *flatMap*; successivamente, vado ad effettuare il controllo con il dizionario creato in precedenza ed applico il POS tagging; dopodichè applico i due filtri e successivamente (sempre attraverso un'altra *filter*), vado a selezionare solamente i tag che fanno riferimento ai sostantivi. Infine, effettuo una wordcount.


In [ ]:
filtered = spark_df.select("cleaned_titles_abstracts").rdd \
            .flatMap(lambda x: x[0].split(' ')) \
            .filter(lambda x: x.lower() not in dictionary) \
            .flatMap(lambda x: nltk.pos_tag([x]))\
            .filter(lambda x: x[0].isdigit() == False) \
            .filter(lambda x: len(x[0]) > 3) \
            .filter(lambda x: x[1] == 'NN' or x[1] == 'NNS' or x[1] == 'NNP' or x[1] == 'NNPS') \
            .map(lambda x: (x, 1)) \
            .reduceByKey(lambda x,y: x+y) \
            .sortBy(ascending=False, keyfunc=lambda key_value: key_value[1]) \
            .collect()

In [ ]:
filtered

[(('measles', 'NNS'), 2174),
 (('rubella', 'NN'), 1785),
 (('mumps', 'NNS'), 1412),
 (('varicella', 'NN'), 1373),
 (('polio', 'NN'), 1130),
 (('tuberculosis', 'NN'), 1078),
 (('hepatitis', 'NN'), 909),
 (('Ebola', 'NN'), 835),
 (('tetanus', 'NN'), 825),
 (('typhus', 'NN'), 574),
 (('months', 'NNS'), 514),
 (('surveillance', 'NN'), 500),
 (('coverage', 'NN'), 496),
 (('samples', 'NNS'), 433),
 (('number', 'NN'), 413),
 (('CONCLUSIONS', 'NN'), 396),
 (('fever', 'NN'), 392),
 (('adults', 'NNS'), 383),
 (('rates', 'NNS'), 380),
 (('zoster', 'NN'), 380),
 (('factors', 'NNS'), 379),
 (('found', 'NN'), 374),
 (('cell', 'NN'), 370),
 (('acute', 'NN'), 341),
 (('elimination', 'NN'), 339),
 (('Measles', 'NNS'), 338),
 (('model', 'NN'), 338),
 (('individuals', 'NNS'), 335),
 (('diagnosis', 'NN'), 327),
 (('review', 'NN'), 324),
 (('laboratory', 'NN'), 324),
 (('cells', 'NNS'), 321),
 (('public', 'NN'), 320),
 (('responses', 'NNS'), 296),
 (('period', 'NN'), 294),
 (('care', 'NN'), 287),
 (('incre

In questo modo, le parole più frequenti sono le malattie.


A questo punto, vado ad effettuare queste trasformazioni che permettono di prelevare i primi 10 elementi, già ordinati per frequenza in precedenza 

In [ ]:
diseases = sc.parallelize(filtered) \
    .map(lambda x: x[0][0].capitalize()) \
    .take(10)

In [ ]:
diseases

['Measles',
 'Rubella',
 'Mumps',
 'Varicella',
 'Polio',
 'Tuberculosis',
 'Hepatitis',
 'Ebola',
 'Tetanus',
 'Typhus']

Adesso, avendo a disposizione la lista delle malattie, possiamo andare ad effettuare delle ricerche in **NCBI**, andando a ricercare dei geni associati alla malattia:
in particolare, vado a ricercare tutti gli id e da questi estraggo il primo, in quanto vengono ordinati per rilevanza.
In particolare, la query che viene effettuata nel database è : ***nome_malattia AND complete genome[title]***

<br>
Dopodichè, salvo il file ottenuto.

In [ ]:
terms = iter(diseases)

for disease in terms:
    search_term = f'{disease} AND complete genome[title]'
    Entrez.email = 'random@example.com'
    handle = Entrez.esearch(db='nucleotide', term=search_term,sort='relevance')
    try:
        genome_id = Entrez.read(handle)['IdList'][0]
    except:
        disease = next(terms)
    record = Entrez.efetch(db="nucleotide", id=genome_id, rettype="gb", retmode="text",sort='relevance')
    filename = f"{'GENBANK'}/{disease}_{genome_id}.gb"
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    print('Writing: {}'.format(filename))
    with open(filename, 'w') as f:
        f.write(record.read())

Writing: GENBANK/Measles_9626945.gb
Writing: GENBANK/Rubella_336284682.gb
Writing: GENBANK/Mumps_338784246.gb
Writing: GENBANK/Varicella_365753294.gb
Writing: GENBANK/Polio_1830670945.gb
Writing: GENBANK/Tuberculosis_1190665804.gb
Writing: GENBANK/Hepatitis_22129792.gb
Writing: GENBANK/Ebola_10141003.gb
Writing: GENBANK/Tetanus_1368665179.gb
Writing: GENBANK/Typhus_478437796.gb


Infine, è possibile visualizzare i vari file *GenBank*

In [ ]:
%%bash
cat GENBANK/Measles_9626945.gb

LOCUS       NC_001498              15894 bp    cRNA    linear   VRL 13-AUG-2018
DEFINITION  Measles virus, complete genome.
ACCESSION   NC_001498
VERSION     NC_001498.1
DBLINK      Project: 15025
            BioProject: PRJNA485481
KEYWORDS    RefSeq.
SOURCE      Measles morbillivirus
  ORGANISM  Measles morbillivirus
            Viruses; Riboviria; Orthornavirae; Negarnaviricota;
            Haploviricotina; Monjiviricetes; Mononegavirales; Paramyxoviridae;
            Orthoparamyxovirinae; Morbillivirus.
REFERENCE   1  (sites)
  AUTHORS   Rima,B.K. and Duprex,W.P.
  TITLE     The measles virus replication cycle
  JOURNAL   Curr. Top. Microbiol. Immunol. 329, 77-102 (2009)
   PUBMED   19198563
REFERENCE   2
  AUTHORS   Takeuchi,K., Miyajima,N., Kobune,F. and Tashiro,M.
  TITLE     Comparative nucleotide sequence analyses of the entire genomes of
            B95a cell-isolated and vero cell-isolated measles viruses from the
            same patient
  JOURNAL   Virus Genes 20 (3), 253-

In [ ]:
for filename in os.listdir('GENBANK'):
    for gb_record in SeqIO.parse(open('GENBANK/'+filename,"r"), "genbank"):
        os.makedirs(os.path.dirname('GENES/'), exist_ok=True)
for word in diseases:
    with open('GENES/'+word+'.txt', 'w') as f:
        f.write('NAME :'+gb_record.name + '\n')
        f.write('DESCRIPTION :'+gb_record.description+ '\n')
        f.write(str(gb_record.seq))

E i genomi dei vari agenti eziologici delle malattie.

In [ ]:
for filename in os.listdir('GENBANK'):
    for gb_record in SeqIO.parse(open('GENBANK/'+filename,"r"), "genbank"):
        os.makedirs(os.path.dirname('GENES/'), exist_ok=True)
        with open('GENES/' + filename.split('_')[0] + '.txt','w') as f:
            print('Writing: {}'.format(filename.split('_')[0] + '.txt'))
            f.write('NAME: '+gb_record.name + '\n')
            f.write('DESCRIPTION: '+gb_record.description+ '\n')
            f.write(str(gb_record.seq))

Writing: Mumps.txt
Writing: Hepatitis.txt
Writing: Ebola.txt
Writing: Typhus.txt
Writing: Rubella.txt
Writing: Tetanus.txt
Writing: Tuberculosis.txt
Writing: Polio.txt
Writing: Measles.txt
Writing: Varicella.txt


In [ ]:
%%bash
cat GENES/Ebola.txt

NAME: AF086833
DESCRIPTION: Ebola virus - Mayinga, Zaire, 1976, complete genome
CGGACACACAAAAAGAAAGAAGAATTTTTAGGATCTTTTGTGTGCGAATAACTATGAGGAAGATTAATAATTTTCCTCTCATTGAAATTTATATCGGAATTTAAATTGAAATTGTTACTGTAATCACACCTGGTTTGTTTCAGAGCCACATCACAAAGATAGAGAACAACCTAGGTCTCCGAAGGGAGCAAGGGCATCAGTGTGCTCAGTTGAAAATCCCTTGTCAACACCTAGGTCTTATCACATCACAAGTTCCACCTCAGACTCTGCAGGGTGATCCAACAACCTTAATAGAAACATTATTGTTAAAGGACAGCATTAGTTCACAGTCAAACAAGCAAGATTGAGAATTAACCTTGGTTTTGAACTTGAACACTTAGGGGATTGAAGATTCAACAACCCTAAAGCTTGGGGTAAAACATTGGAAATAGTTAAAAGACAAATTGCTCGGAATCACAAAATTCCGAGTATGGATTCTCGTCCTCAGAAAATCTGGATGGCGCCGAGTCTCACTGAATCTGACATGGATTACCACAAGATCTTGACAGCAGGTCTGTCCGTTCAACAGGGGATTGTTCGGCAAAGAGTCATCCCAGTGTATCAAGTAAACAATCTTGAAGAAATTTGCCAACTTATCATACAGGCCTTTGAAGCAGGTGTTGATTTTCAAGAGAGTGCGGACAGTTTCCTTCTCATGCTTTGTCTTCATCATGCGTACCAGGGAGATTACAAACTTTTCTTGGAAAGTGGCGCAGTCAAGTATTTGGAAGGGCACGGGTTCCGTTTTGAAGTCAAGAAGCGTGATGGAGTGAAGCGCCTTGAGGAATTGCTGCCAGCAGTATCTAGTGGAAAAAACATTAAGAGAACACTTGCTGCCATGCCGGAAGAGGAGACAACTGAAGCTAATGCCGGTCAGTTTC